In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd

from detector import Detector
from util import load_image
import os
import ipdb

weight_path = '/data/Denoise.pickle'
model_path = '../models/na/'
pretrained_model_path = '' #"/media/data3/thang/home/PACHA/Weakly_detector/models/Denoise/model-50"
n_epochs = 10
init_learning_rate = 0.00001
weight_decay_rate = 0.0005
momentum = 0.9
batch_size = 1

In [0]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [0]:
sys.version_info

In [26]:
import cv2
img = cv2.imread('/content/drive/My Drive/CAM/inputs/na/Train_set/1.Variant-1/C 24A-line10-cr-letter20.png')
img

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [0]:
import zipfile
import io
data = zipfile.ZipFile('na.zip', 'r')
data.extractall()

In [0]:
dataset_path = '../inputs/na'
#dataset_path = '../inputs/Denoise'

cham_train_path = '../data/na/Train_set'
cham_test_path = '../data/na/Test_set'
trainset_path = '../data/na/train.pickle'
testset_path = '../data/na/test.pickle'
label_dict_path = '../data/na/label_dict.pickle'

extensions = set(['.png','.jpg'])

if not os.path.exists( trainset_path ):
    if not os.path.exists( cham_train_path ):
        os.makedirs( cham_train_path )
        os.makedirs( cham_test_path)
    train_dir_list = os.listdir( dataset_path + '/Train_set' )
    test_dir_list = os.listdir( dataset_path + '/Test_set' )
    label_pairs = map(lambda x: x.split('.'), train_dir_list)
    labels, label_names = zip(*label_pairs)
    labels = map(lambda x: int(x), labels)
    label_dict = pd.Series( labels, index=label_names )
    label_dict -= 1 # starting from 0
    n_labels = len( label_dict )

    train_paths_per_label = list(map(lambda one_dir: map(lambda one_file: os.path.join( dataset_path + '/Train_set', one_dir, one_file ), os.listdir( os.path.join( dataset_path + '/Train_set', one_dir))), train_dir_list))
    test_paths_per_label = list(map(lambda one_dir: map(lambda one_file: os.path.join( dataset_path + '/Test_set', one_dir, one_file ), os.listdir( os.path.join( dataset_path + '/Test_set', one_dir))), test_dir_list))

    image_paths_train = np.hstack(train_paths_per_label)
    image_paths_test = np.hstack(test_paths_per_label)
    
    trainset = pd.DataFrame({'image_path': image_paths_train})
    testset  = pd.DataFrame({'image_path': image_paths_test })
    
    trainset = trainset[ trainset['image_path'].map( lambda x: x.endswith('.png'))]
    trainset['label'] = trainset['image_path'].map(lambda x: int(x.split('/')[-2].split('.')[0])-1)
    trainset['label_name'] = trainset['image_path'].map(lambda x: x.split('/')[-2].split('.')[1])

    testset = testset[ testset['image_path'].map( lambda x: str(x).endswith('.png'))]
    testset['label'] = testset['image_path'].map(lambda x: int(x.split('/')[-2].split('.')[0])-1)
    testset['label_name'] = testset['image_path'].map(lambda x: x.split('/')[-2].split('.')[1])

    label_dict.to_pickle(label_dict_path)
    trainset.to_pickle(trainset_path)
    testset.to_pickle(testset_path)

else:
    trainset = pd.read_pickle( trainset_path )
    testset  = pd.read_pickle( testset_path )
    label_dict = pd.read_pickle( label_dict_path )
    n_labels = len(label_dict)

In [0]:
# https://stackoverflow.com/a/46760323/9815299
trainset['image_path'] = [train_path.replace('/media/data3/thang/home/PACHA/Weakly_detector/inputs/na/Resized/', '../inputs/na/') for train_path in trainset['image_path']]
testset['image_path'] = [testset.replace('/media/data3/thang/home/PACHA/Weakly_detector/inputs/na/Resized/', '../inputs/na/') for testset in testset['image_path']]

In [0]:
trainset.image_path[0]

'/Users/mac/Documents/workspace/Python/PACH-code/CAM/inputs/na/Train_set/4.Variant-4/C 38 D-or-line11-cr-letter1.png'

In [0]:
# Training
# Train loss = nan: https://stackoverflow.com/a/40434284/9815299
learning_rate = tf.placeholder( tf.float32, [])
images_tf = tf.placeholder( tf.float32, [None, 224, 224, 3], name="images")
labels_tf = tf.placeholder( tf.int64, [None], name='laels')

detector = Detector(weight_path, n_labels)

p1,p2,p3,p4,conv5, conv6, gap, output = detector.inference(images_tf)
loss_tf = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits( logits=output, labels=labels_tf ) + 1e-10 )

weights_only = filter( lambda x: x.name.endswith('W:0'), tf.trainable_variables() )
weight_decay = tf.reduce_sum(tf.stack([tf.nn.l2_loss(x) for x in weights_only])) * weight_decay_rate
loss_tf += weight_decay

# Assume that you have 12GB of GPU memory and want to allocate ~4GB:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)

sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
saver = tf.train.Saver( max_to_keep=100 )

optimizer = tf.train.MomentumOptimizer( learning_rate, momentum )
grads_and_vars = optimizer.compute_gradients( loss_tf )
grads_and_vars = map(lambda gv: (gv[0], gv[1]) if ('conv6' in gv[1].name or 'GAP' in gv[1].name) else (gv[0]*0.1, gv[1]), grads_and_vars)
#grads_and_vars = [(tf.clip_by_value(gv[0], -5., 5.), gv[1]) for gv in grads_and_vars]
train_op = optimizer.apply_gradients( grads_and_vars )
tf.global_variables_initializer().run()

if pretrained_model_path:
    print ("Pretrained")
    saver.restore(sess, pretrained_model_path)

testset.index  = range( len(testset) )
#testset = testset.ix[np.random.permutation( len(testset) )]#[:1000]
#trainset2 = testset[1000:]
#testset = testset[:1000]

#trainset = pd.concat( [trainset, trainset2] )
# We lack the number of training set. Let's use some of the test images

f_log = open('../results/log.Denoise.txt', 'w')

iterations = 0
loss_list = []
for epoch in range(n_epochs):

    trainset.index = range( len(trainset) )
    trainset = trainset.ix[ np.random.permutation( len(trainset) )]

    for start, end in zip(
        range( 0, len(trainset)+batch_size, batch_size),
        range(batch_size, len(trainset)+batch_size, batch_size)):

        current_data = trainset[start:end]
        current_image_paths = current_data['image_path'].values
        current_images = np.array(map(lambda x: load_image(x), current_image_paths))
        
        good_index = np.array(map(lambda x: x is not None, current_images))
        #good_index = np.fromiter(map(lambda x: x is not None, current_images), dtype=float)
        current_data = current_data[good_index]
        current_images = np.stack(current_images[good_index])
        current_labels = current_data['label'].values

        _, loss_val, output_val = sess.run(
                [train_op, loss_tf, output],
                feed_dict={
                    learning_rate: init_learning_rate,
                    images_tf: current_images,
                    labels_tf: current_labels
                    })

        loss_list.append( loss_val )

        iterations += 1
        if iterations % 5 == 0:
            print ("======================================")
            print ("Epoch", epoch, "Iteration", iterations)
            print ("Processed", start, '/', len(trainset))

            label_predictions = output_val.argmax(axis=1)
            acc = (label_predictions == current_labels).sum()

            print ("Accuracy:", acc, '/', len(current_labels))
            print ("Training Loss:", np.mean(loss_list))
            print ("\n")
            loss_list = []

    n_correct = 0
    n_data = 0
    for start, end in zip(
            range(0, len(testset)+batch_size, batch_size),
            range(batch_size, len(testset)+batch_size, batch_size)
            ):
        current_data = testset[start:end]
        current_image_paths = current_data['image_path'].values
        current_images = np.array(map(lambda x: load_image(x), current_image_paths))

        good_index = np.array(map(lambda x: x is not None, current_images))

        current_data = current_data[good_index]
        current_images = np.stack(current_images[good_index])
        current_labels = current_data['label'].values

        output_vals = sess.run(
                output,
                feed_dict={images_tf:current_images})

        label_predictions = output_vals.argmax(axis=1)
        acc = (label_predictions == current_labels).sum()

        n_correct += acc
        n_data += len(current_data)

    acc_all = n_correct / float(n_data)
    f_log.write('epoch:'+str(epoch)+'\tacc:'+str(acc_all) + '\n')
    print ("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
    print ('epoch:'+str(epochoch)+'\tacc:'+str(acc_all) + '\n')
    print ("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")

    saver.save( sess, os.path.join( model_path, 'model'), global_step=epoch)

    init_learning_rate *= 0.99

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


TypeError: iteration over a 0-d array

# save weights: https://github.com/google/prettytensor/issues/6#issuecomment-380919368

vars = tf.trainable_variables()
print(vars) #some infos about variables...
vars_vals = sess.run(vars)

new_weights = {}
for var, val in zip(vars, vars_vals):
    #print("var: {}, value: {}".format(var.name, val)) #...or sort it in a list....
    layer_name = str(var.name.split('/')[0])
    
    if layer_name in new_weights:
        # append the new number to the existing array at this slot
        new_weights[layer_name].append(val)
    else:
        # create a new array in this slot
        new_weights[layer_name] = [val]
new_weights

#https://stackoverflow.com/a/11218504/9815299
with open('../data/na_weights.pickle', 'wb') as handle:
    pickle.dump(new_weights, handle, protocol=pickle.HIGHEST_PROTOCOL)